In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

from glob import glob
from tqdm import tqdm
from scipy.io import loadmat
from PIL import Image

import keras 

import seaborn as sns
plt.style.use('seaborn')
sns.set(style="ticks", color_codes=True)

state = 1
np.random.seed(1)


DATA_PATH = '/home/jiehuan/PycharmProjects/Image_edge/Data/BSR500/BSDS500/data'
ROOT_PATH = '/home/jiehuan'
GROUND_PATH = DATA_PATH + '/groundTruth'
ORI_IMAGE_PATH = DATA_PATH + '/images'
OUT_IMAGE_PATH = '/home/jiehuan/PycharmProjects/Image_edge/image'
MODEL_PATH = '/home/jiehuan/PycharmProjects/Image_edge/model'
WEIGHT_PATH = '/home/jiehuan/PycharmProjects/Image_edge/weight'

POSTDATA_PATH = '/home/jiehuan/PycharmProjects/Image_edge/Data/BSR500/BSDS500/data/dataAgumentation'


def save_fig(fig_id, tight_layout=True):
    if not os.path.exists(OUT_IMAGE_PATH):
        os.makedirs(OUT_IMAGE_PATH)
    path = os.path.join(OUT_IMAGE_PATH, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


In [9]:
i=0

for foldname in tqdm(os.listdir(ORI_IMAGE_PATH)):
    if foldname == 'val' or foldname == 'train':
        for img in sorted(glob(ORI_IMAGE_PATH + '/' + foldname + '/*')):
            data = cv2.imread(img)
            if data is not None:
                shape = np.asarray(data).shape
                (filepath,tempfilename) = os.path.split(img)
                (filename,extension) = os.path.splitext(tempfilename)

                if shape == (481, 321, 3): #height width
                    out0 = cv2.resize(data, (480, 320))
                    data = cv2.resize(data, (320, 480)) #width height
                    data = Image.fromarray(data)
                    data = np.asarray(data.rotate(90, expand=True))
                    newname0 = POSTDATA_PATH +"/train/"+filename+"S.npy"
                    np.save(newname0, out0) # extension + S
                else:
                    data = cv2.resize(data, (480, 320))
                
                
                rdata = Image.fromarray(data).rotate(3, expand=0)
                box = (10,10,470,310) #(left，up，right，down)
                region = rdata.crop(box)
                out2 = np.asarray(region.resize((480,320)))
                
                rdata = Image.fromarray(data).rotate(1, expand=0)
                box = (3,3,477,317) #(left，up，right，down)
                region = rdata.crop(box)
                out3 = np.asarray(region.resize((480,320)))
                
                
                if i<271:
                    orgdata  = POSTDATA_PATH +"/train/"+filename+".npy"                
                    newname1 = POSTDATA_PATH +"/train/"+filename+"T.npy"
                    newname2 = POSTDATA_PATH +"/train/"+filename+"R1.npy"
                    newname3 = POSTDATA_PATH +"/train/"+filename+"R2.npy"
                else:
                    orgdata  = POSTDATA_PATH +"/val/"+filename+".npy"
                    newname1 = POSTDATA_PATH +"/val/"+filename+"T.npy"
                    newname2 = POSTDATA_PATH +"/val/"+filename+"R1.npy"
                    newname3 = POSTDATA_PATH +"/val/"+filename+"R2.npy"
                
                out1 = np.fliplr(data)
                np.save(orgdata, data)      # original Data
                np.save(newname1, out1)     # horizontal flip + T
                np.save(newname2, out2)     # rotate(3) and chop (10,10,470,310)
                np.save(newname3, out3)     # rotate(1) and chop (3,3,477,317)
            i +=1



100%|██████████| 3/3 [00:09<00:00,  3.10s/it]


In [25]:
j=0

for foldname in os.listdir(GROUND_PATH):
    for img in sorted(os.listdir(os.path.join(GROUND_PATH, foldname))):
        (filepath,tempfilename) = os.path.split(img)
        (filename,extension) = os.path.splitext(tempfilename)
        IMG_DIR = os.path.join(GROUND_PATH, foldname, img)
        mat = loadmat(IMG_DIR)
        matdata = mat['groundTruth']
        if matdata[0, 0]['Boundaries'][0, 0].shape == (321, 481):
            data_c = np.zeros((321, 481))
        else:
            data_c = np.zeros((481, 321))
        #method for selection of ground truth
        for i in range(0, matdata.shape[1]):
            data_m = matdata[0, i]['Boundaries'][0, 0]
            data_c = np.maximum(data_m, data_c)
        #above for loop needs to be update
        if data_c.shape == (481, 321):  # size: H - W
            data = cv2.resize(data_c, (320, 480))  # size: W - H
            data = Image.fromarray(data)
            data = np.asarray(data.rotate(90, expand=True))
            newname0 = POSTDATA_PATH +"/groundtruth/train/"+filename+"S.npy"
            np.save(newname0, out0) # extension + S
        else:
            data = cv2.resize(data_c, (480, 320))
        
        rdata = Image.fromarray(data).rotate(3, expand=0)
        box = (10,10,470,310) #(left，up，right，down)
        region = rdata.crop(box)
        out2 = np.asarray(region.resize((480,320)))
                
        rdata = Image.fromarray(data).rotate(1, expand=0)
        box = (3,3,477,317) #(left，up，right，down)
        region = rdata.crop(box)
        out3 = np.asarray(region.resize((480,320)))        
        
        
        if j<271:
            orgdata  = POSTDATA_PATH +"/groundtruth/train/"+filename+".npy"                
            newname1 = POSTDATA_PATH +"/groundtruth/train/"+filename+"T.npy"
            newname2 = POSTDATA_PATH +"/groundtruth/train/"+filename+"R1.npy"
            newname3 = POSTDATA_PATH +"/groundtruth/train/"+filename+"R2.npy"
        else:
            orgdata  = POSTDATA_PATH +"/groundtruth/val/"+filename+".npy"
            newname1 = POSTDATA_PATH +"/groundtruth/val/"+filename+"T.npy"
            newname2 = POSTDATA_PATH +"/groundtruth/val/"+filename+"R1.npy"
            newname3 = POSTDATA_PATH +"/groundtruth/val/"+filename+"R2.npy"
                
        out1 = np.fliplr(data)
        np.save(orgdata, data)      # original Data
        np.save(newname1, out1)     # horizontal flip + T
        np.save(newname2, out2)     # rotate(3) and chop (10,10,470,310)
        np.save(newname3, out3)     # rotate(1) and chop (3,3,477,317)
        j +=1

            
            
            